# RL Exercise 5 - Summarizing Text Using RL

**GOAL:** The goal of this exercise is to show how one might frame the problem of text summarization as a reinforcement learning problem and train a policy to perform text summarization.

Text summarization here means taking some writing and shortening it.

**NOTE:** In this example, we will use an extremely **simplistic** policy for *extractive summarization*. Extractive summarization involves summarizing text by keeping some subset of the original text and discarding the rest. In contrast, *abstractive summarization* involves synthesizing brand new text, and is required for producing high quality summaries.

## Problem Setup

We set up the problem as follows.

1. The **state** of the environment consists of pre-trained vectors for the current sentence from the original text and several of the previous and subsequent sentences. One way this setup could be improved is by putting much more information in the state so that the policy can make a more informed decision.
2. The **action** that can be taken by the policy is to either include the current sentence in the summary or to drop the current sentence from the summary.
3. The state of the environment starts at the first sentence in the original text. After an action is taken, the state transitions to the next sentence in the original text.
4. At each time step, the **reward** is the incremental increase in the [ROUGE score][1] obtained by the current partial summarization (relative to the partial summarization from the previous time step). Computing this requires a ground truth summary.

[1]: https://en.wikipedia.org/wiki/ROUGE_(metric)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import summarization
import ray

from ray.rllib.ppo import PPOAgent, DEFAULT_CONFIG

Start Ray.

In [ ]:
ray.init(num_workers=0)

The cell below is a hack. The explanation is as follows. Internally within the `PPOAgent` constructor, a number of actors are created, and these actors will instantiate gym environments using the command `gym.make('SimpleSummarization-v0')`. The command `gym.make` knows how to instantiate a number of pre-defined environments that are shipped with the `gym` module. However, the `SimpleSummarization-v0` environment is defined in the `summarization` module and is registered with the `gym` module when the `import summarization` statement gets run.

Therefore, for the actors to successfully instantiate the gym environments, the `summarization` module must be imported on the actors. This is why we define a remote function `import_summarization` which closes over the `summarization` environment. When the actors are created, that remote function is unpickled on the actors which forces the `summarization` module to be imported, which enables the `gym` module to create the `SimpleSummarization-v0` environment.

In [ ]:
# This is a hack.
@ray.remote
def import_summarization():
    summarization

Instantiate an agent that can be trained using Proximal Policy Optimization (PPO).

In [ ]:
config = DEFAULT_CONFIG.copy()
# Consider using more workers to speed up the rollouts.
config['num_workers'] = 10
config['gamma'] = 0.99
config['sgd_stepsize'] = 5e-3
config['kl_coeff'] = 0.1
config['num_sgd_iter'] = 20
config['sgd_batchsize'] = 8196
config['observation_filter'] = 'NoFilter'
config['model']['fcnet_hiddens'] = [32, 32]

agent = PPOAgent('SimpleSummarization-v0', config)

**EXERCISE:** Train the policy for several iterations. You can ignore this for right now and experiment below with the untrained policy. Then try training the policy and see if the summarizations improve.

In [ ]:
for i in range(2):
    result = agent.train()

## Generate Summaries

Try using the policy to generate an extractive summary of a sentence. You can read the summary by running `print(summary)`.

**NOTE:** This exercise performs training with very small amounts of data, very little time, and a very simplistic model. To perform well, these applications require more data, more time, and more sophisticated models.

In [ ]:
document = """
Life is short, as everyone knows. When I was a kid I used to wonder about this. Is life actually short, or are we really complaining about its finiteness? Would we be just as likely to feel life was short if we lived 10 times as long? Since there didn't seem any way to answer this question, I stopped wondering about it. Then I had kids. That gave me a way to answer the question, and the answer is that life actually is short. Having kids showed me how to convert a continuous quantity, time, into discrete quantities. You only get 52 weekends with your 2 year old. If Christmas-as-magic lasts from say ages 3 to 10, you only get to watch your child experience it 8 times. And while it's impossible to say what is a lot or a little of a continuous quantity like time, 8 is not a lot of something. If you had a handful of 8 peanuts, or a shelf of 8 books to choose from, the quantity would definitely seem limited, no matter what your lifespan was. Ok, so life actually is short. Does it make any difference to know that? It has for me. It means arguments of the form "Life is too short for x" have great force. It's not just a figure of speech to say that life is too short for something. It's not just a synonym for annoying. If you find yourself thinking that life is too short for something, you should try to eliminate it if you can. When I ask myself what I've found life is too short for, the word that pops into my head is "bullshit." I realize that answer is somewhat tautological. It's almost the definition of bullshit that it's the stuff that life is too short for. And yet bullshit does have a distinctive character. There's something fake about it. It's the junk food of experience. If you ask yourself what you spend your time on that's bullshit, you probably already know the answer. Unnecessary meetings, pointless disputes, bureaucracy, posturing, dealing with other people's mistakes, traffic jams, addictive but unrewarding pastimes. There are two ways this kind of thing gets into your life: it's either forced on you or it tricks you. To some extent you have to put up with the bullshit forced on you by circumstances. You need to make money, and making money consists mostly of errands. Indeed, the law of supply and demand insures that: the more rewarding some kind of work is, the cheaper people will do it. It may be that less bullshit is forced on you than you think, though. There has always been a stream of people who opt out of the default grind and go live somewhere where opportunities are fewer in the conventional sense, but life feels more authentic. This could become more common. You can do it on a smaller scale without moving. The amount of time you have to spend on bullshit varies between employers. Most large organizations (and many small ones) are steeped in it. But if you consciously prioritize bullshit avoidance over other factors like money and prestige, you can probably find employers that will waste less of your time. If you're a freelancer or a small company, you can do this at the level of individual customers. If you fire or avoid toxic customers, you can decrease the amount of bullshit in your life by more than you decrease your income. But while some amount of bullshit is inevitably forced on you, the bullshit that sneaks into your life by tricking you is no one's fault but your own. And yet the bullshit you choose may be harder to eliminate than the bullshit that's forced on you. Things that lure you into wasting your time on them have to be really good at tricking you. An example that will be familiar to a lot of people is arguing online. When someone contradicts you, they're in a sense attacking you. Sometimes pretty overtly. Your instinct when attacked is to defend yourself. But like a lot of instincts, this one wasn't designed for the world we now live in. Counterintuitive as it feels, it's better most of the time not to defend yourself. Otherwise these people are literally taking your life. Arguing online is only incidentally addictive. There are more dangerous things than that. As I've written before, one byproduct of technical progress is that things we like tend to become more addictive. Which means we will increasingly have to make a conscious effort to avoid addictions—to stand outside ourselves and ask "is this how I want to be spending my time?" As well as avoiding bullshit one should actively seek out things that matter. But different things matter to different people, and most have to learn what matters to them. A few are lucky and realize early on that they love math or taking care of animals or writing, and then figure out a way to spend a lot of time doing it. But most people start out with a life that's a mix of things that matter and things that don't, and only gradually learn to distinguish between them. For the young especially, much of this confusion is induced by the artificial situations they find themselves in. In middle school and high school, what the other kids think of you seems the most important thing in the world. But when you ask adults what they got wrong at that age, nearly all say they cared too much what other kids thought of them. One heuristic for distinguishing stuff that matters is to ask yourself whether you'll care about it in the future. Fake stuff that matters usually has a sharp peak of seeming to matter. That's how it tricks you. The area under the curve is small, but its shape jabs into your consciousness like a pin. The things that matter aren't necessarily the ones people would call "important." Having coffee with a friend matters. You won't feel later like that was a waste of time. One great thing about having small children is that they make you spend time on things that matter: them. They grab your sleeve as you're staring at your phone and say "will you play with me?" And odds are that is in fact the bullshit-minimizing option. If life is short, we should expect its shortness to take us by surprise. And that is just what tends to happen. You take things for granted, and then they're gone. You think you can always write that book, or climb that mountain, or whatever, and then you realize the window has closed. The saddest windows close when other people die. Their lives are short too. After my mother died, I wished I'd spent more time with her. I lived as if she'd always be there. And in her typical quiet way she encouraged that illusion. But an illusion it was. I think a lot of people make the same mistake I did. The usual way to avoid being taken by surprise by something is to be consciously aware of it. Back when life was more precarious, people used to be aware of death to a degree that would now seem a bit morbid. I'm not sure why, but it doesn't seem the right answer to be constantly reminding oneself of the grim reaper hovering at everyone's shoulder. Perhaps a better solution is to look at the problem from the other end. Cultivate a habit of impatience about the things you most want to do. Don't wait before climbing that mountain or writing that book or visiting your mother. You don't need to be constantly reminding yourself why you shouldn't wait. Just don't wait. I can think of two more things one does when one doesn't have much of something: try to get more of it, and savor what one has. Both make sense here. How you live affects how long you live. Most people could do better. Me among them. But you can probably get even more effect by paying closer attention to the time you have. It's easy to let the days rush by. The "flow" that imaginative people love so much has a darker cousin that prevents you from pausing to savor life amid the daily slurry of errands and alarms. One of the most striking things I've read was not in a book, but the title of one: James Salter's Burning the Days. It is possible to slow time somewhat. I've gotten better at it. Kids help. When you have small children, there are a lot of moments so perfect that you can't help noticing. It does help too to feel that you've squeezed everything out of some experience. The reason I'm sad about my mother is not just that I miss her but that I think of all the things we could have done that we didn't. My oldest son will be 7 soon. And while I miss the 3 year old version of him, I at least don't have any regrets over what might have been. We had the best time a daddy and a 3 year old ever had. Relentlessly prune bullshit, don't wait to do things that matter, and savor the time you have. That's what you do when life is short.
"""

summary = summarization.summarize(agent, document)

print("Original document length is {}".format(len(document)))
print("Summary length is {}".format(len(summary)))